## 2. Data Understanding

The data used in this project was obtained from the [Zillow housing data](https://www.zillow.com/research/data/). Our aim is to investigate the data in an attempt to get a deeper understanding of it. 



#### 2.1 Data Description

##### Importing the necessary libaries

In [ ]:
# For data manipulation 
import pandas as pd
# For data analysis
import numpy as np
#For data visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Warning libraries
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')


##### Loading the dataset

In [ ]:
data = pd.read_csv("zillow_data.csv")

##### Displaying First Rows

In [ ]:
data.head()

##### Displaying last five rows

In [ ]:
data.tail()

The number of columns(272) in the last and first five rows is the same implying consistency in our dataset

In [ ]:
# Checking the shape of dataset
print('Number of rows:',data.shape[0])
print('Number of columns:',data.shape[1])

The dataset has 14723 rows and 272 columns. 

In [ ]:
# Checking for Datatypes
data.info()

In [ ]:
# Checking Columns
data.columns

In [ ]:
# Description of the columns
columns = {'Columns': ['RegionID','RegionName','City','State','Metro','County Name','Size Rank','Date Columns (265 Columns)'],
'Description':['Unique region identifier','Names of the Regions (Zipcodes)','City names for the regions',
               'Names of the states','Names of metropolitan areas','Names of counties','Rank of Zipcodes by urbanization',
              'Median house prices across the years']}
pd.DataFrame(columns)

In [ ]:
# Summary Statistics
data.describe().T

#### 2.2 Data Quality from the data description
* The dataset has 14723 rows and 272 columns. 
* The columns have both categorical(4) and numerical data(268)
* There are many columns because it is in a wide fromat; the last 265 columns describe the dates of the housing data. 



## 3. Data Preparation



Inorder to enhance the efficiency of our model, the data has to be inspected and cleaned to align with our objectives. 
This is to ensure that we do not generate any misleading information from the analysis. This includes Checking for the validity, consistency, completeness and uniformity. 

#### 3.1 Data Cleaning

#### 3.1.1 Completeness

*  Checking and Handling for missing values


In [ ]:
# Check for missing values 
print(f'The data has {data.isna().sum().sum()} missing values')

In [ ]:
print(f'The following columns contain these missing values {data.isna().sum().sort_values(ascending = False)}')

In [ ]:
per_missing_vals = (data.isna().sum())*100/len(data)
per_missing_vals.sort_values(ascending=False)

In [ ]:
# Handling the missing values
## Fill the `metro` column with the word "missing"
data['Metro'].fillna('missing', inplace=True)

In [ ]:
missing_values = data.isna().sum().sort_values(ascending=False)
percent = missing_values*100/len(data)
percent.sort_values(ascending=False)
percent

In [ ]:
## Handling the date columns' missing values
data.interpolate(inplace=True)

In [ ]:
#Check for the missing values again
data.isna().sum().value_counts()

#### 3.1.2 Validity
* Checking for duplicated values 
* Checking for Outliers in the dataset


In [ ]:
# Checking for duplicated values  
data.duplicated().sum()


There are no duplicated values in the dataset

We  do not check for outliers in the data, as their presence helps improve the accuracy of the model since in real life there are houses that are priced highly above and below the average. 

#### 3.1.3 Uniformity
* Checking if the column names are uniform 
* Exploring the columns more
* Checking if the data types are relevant to the column description


In [ ]:
# Cheking the column names again 
data.columns

In [ ]:
# Checking the data type of the RegionName column 
data.dtypes['RegionName']

In [ ]:
# Renaming the RegionName column to Zipcode because the column's data is in numerical form
data.rename(columns = {'RegionName': "Zipcode"}, inplace=True)

In [ ]:
# Changing the data type of the Zipcode column to categorical  
data.Zipcode = data.Zipcode.astype('string')

In [ ]:
# Checking for the unique values in the Zipcode column
data.Zipcode.nunique()


#### Data Construction

Here we will  derive new attributes from the data that will be helpful in answering our research questions.

In [ ]:

# calculating and creating a new column -ROI

data['ROI'] = (data['2018-04']/ data['1996-04'])-1



#calculating std to be used to find CV
data["std"] = data.loc[:, "1996-04":"2018-04"].std(skipna=True, axis=1)

#calculating mean to be used to find CV
data["mean"] = data.loc[:, "1996-04":"2018-04"].mean(skipna=True, axis=1)

# calculating and creating a new column - CV

data["CV"] = data['std']/data["mean"]

# dropping std and mean as they are not necessary for analysis

data.drop(["std", "mean"], inplace=True, axis=1)

In [ ]:
data.columns

#### Convert data to Time Series

In [ ]:
# Create a copy of the dataset to convert into long view while preserving df as a wide view for EDA
new_data = data.copy()

In [ ]:
# creating a function that changes the dataframe structure from wide view to long view

def melt_df(data):
    melted = pd.melt(data, id_vars=['RegionID','Zipcode', 'City', 'State', 'Metro', 'CountyName', 'SizeRank','ROI','CV'], var_name='Date')
    melted['Date'] = pd.to_datetime(melted['Date'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [ ]:
# Loading the melted dataset

new_data = melt_df(new_data)


In [ ]:
# First five rows of the melted dataset

new_data.head()

In [ ]:
# Bottom five rows of the melted dataset

new_data.tail()

In [ ]:
# Number of rows and columns

print('Number of rows:',new_data.shape[0])
print('Number of columns:',new_data.shape[1])

In [ ]:
# Numerical Columns
print(f"Numerical Columns: {new_data.select_dtypes(include='int').columns}\n")

# Categorical Columns
print(f"Categorical Columns: {new_data.select_dtypes(include='object').columns}")

In [ ]:
# Summary Statistics
new_data.describe()

##### Data Indexing 
While working with time series data in Python, having dates (or datetimes) in the index can be very helpful, especially if they are of DatetimeIndex type.

In [ ]:
## Making the date column into the index

new_data.set_index('Date',inplace = True)


In [ ]:
# Checking to confirm if the index column of our new dataframe is Date column 
if isinstance(new_data.index, pd.DatetimeIndex):
    new_data.index.name == 'Date'
    print("Index column is date!")
else:
    print('Index column is not Date')

In [ ]:
# Inspecting Index column 
new_data.index

The output above shows that our dataset clearly fulfills the indexing requirements. Look at the last line:

dtype='datetime64[ns]',... length=3901595,...'

dtype=datetime[ns] field confirms that the index is made of timestamp objects.
length=3901595 shows the total number of entries in our time series data.


#### Exploratory Data Analysis

 Exploration will be done on the data to determine:

- Does Urbanization Affect Median House Prices?

- Which cities fetch the highest median house prices?

- What top 5 Zipcodes have the highest ROI?

- Which zipcodes have high price volatility?



#### Time Series EDA

This analysis will answer the question: What is the trend of median houseprices over the years?
 
However, given the large volume of our data, we perform our EDA in two phases: one between 1996 to 2007 and another between 2007 and 2018.

In [ ]:
# Slicing the data

time_series1 = new_data['1996-04-01':'2007-12-31']
time_series2 = new_data['2007-01-01':'2018-04-01']


#### Series 1

In [ ]:
time_series1_monthly = time_series1.resample('MS').mean()['value']
# Draw a line plot using the new data
time_series1_monthly.plot(figsize = (22,8))

plt.title('Typical Home Value by Month Between 1996 and 2007',fontsize = 20)
plt.ylabel('Value in US Dollars ($)',fontsize = 20)
plt.xlabel ('1996 - 2007',fontsize = 20)

plt.yticks(fontsize = 20)
plt.xticks (fontsize = 20)

plt.show()

In [ ]:
# Annual 
yearly_data =  time_series1['value'].resample(rule='A').mean()
yearly_data.plot.line(color='magenta',)

plt.title('Home Value by Year')
plt.ylabel('Value in US Dollars ($)')
plt.xlabel ('1996 - 2007')


plt.show()

In [ ]:
# Plot a histogram of the dataset
time_series1_monthly.hist(figsize = (12,6))
plt.show()

In [ ]:
# Plot a density plot for temperature dataset
time_series1_monthly.plot(kind='kde', figsize = (12,6))
plt.show()

#### Series 2

In [ ]:
time_series2_monthly = time_series2.resample('MS').mean()['value']
# Draw a line plot using the new data
time_series2_monthly.plot(figsize = (22,8))

plt.title('Typical Home Value by Month Between 2007 and April 2018',fontsize = 20)
plt.ylabel('Value in US Dollars ($)',fontsize = 20)
plt.xlabel ('2007 -2018',fontsize = 20)

plt.yticks(fontsize = 20)
plt.xticks (fontsize = 20)

plt.show()

In [ ]:
# Annual 
yearly_data =  time_series2['value'].resample(rule='A').mean()
yearly_data.plot.line(color='magenta',)

plt.title('Home Value by Year')
plt.ylabel('Value in US Dollars ($)')
plt.xlabel ('2007 - 2018')


plt.show()

In [ ]:
# Plot a histogram of the dataset
time_series2_monthly.hist(figsize = (12,6))
plt.show()

In [ ]:
# Plot a density plot for the dataset
time_series2_monthly.plot(kind='kde', figsize = (12,6))
plt.show()

#### Heatmap for both Series

In [ ]:
# Transpose the yearly group DataFrame
year_matrix = new_data['value'].resample(rule='A').mean().to_frame().T

# Draw a heatmap with matshow()
plt.matshow( year_matrix, interpolation=None, aspect='auto', cmap=plt.cm.Spectral_r)
plt.show()